 Create a Loop to Fine-Tune All Models


In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset
import evaluate
import numpy as np
import time

def read_conll(file_path):
    sentences = []
    tokens, labels = [], []

    with open(file_path, encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == "":
                if tokens:
                    sentences.append({"tokens": tokens, "ner_tags": labels})
                    tokens, labels = [], []
            else:
                splits = line.split('\t')
                if len(splits) == 2:
                    token, label = splits
                    tokens.append(token)
                    labels.append(label)

    return sentences

# Load and split once
dataset = Dataset.from_list(read_conll("ner_amharic_sample.conll"))
dataset = dataset.train_test_split(test_size=0.2)

# Reuse label mapping
label_list = sorted(set(tag for row in dataset['train'] for tag in row["ner_tags"]))
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

In [2]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


Define Training and Evaluation Function

In [14]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset
import evaluate
import numpy as np
import time
from seqeval.metrics import get_entities # Import get_entities

def read_conll(file_path):
    sentences = []
    tokens, labels = [], []

    with open(file_path, encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == "":
                if tokens:
                    sentences.append({"tokens": tokens, "ner_tags": labels})
                    tokens, labels = [], []
            else:
                splits = line.split('\t')
                if len(splits) == 2:
                    token, label = splits
                    tokens.append(token)
                    labels.append(label)

    return sentences

# Load and split once
# dataset = Dataset.from_list(read_conll("ner_amharic_sample.conll"))
# dataset = dataset.train_test_split(test_size=0.2)

# Reuse label mapping
# label_list = sorted(set(tag for row in dataset['train'] for tag in row["ner_tags"]))
# label2id = {l: i for i, l in enumerate(label_list)}
# id2label = {i: l for l, i in label2id.items()}


def fine_tune_model(model_checkpoint, model_name, dataset):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    def align_labels_with_tokens(example):
        tokenized_inputs = tokenizer(example["tokens"], is_split_into_words=True, truncation=True)
        word_ids = tokenized_inputs.word_ids()
        aligned_labels = []
        previous_word = None
        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)
            elif word_id != previous_word:
                aligned_labels.append(label2id[example["ner_tags"][word_id]])
            else:
                aligned_labels.append(label2id[example["ner_tags"][word_id]] if example["ner_tags"][word_id].startswith("I-") else -100)
            previous_word = word_id
        tokenized_inputs["labels"] = aligned_labels
        return tokenized_inputs

    tokenized_dataset = dataset.map(align_labels_with_tokens, batched=True) # Added batched=True

    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id
    )

    args = TrainingArguments(
        output_dir=f"results/{model_name}",
        eval_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=5,
        weight_decay=0.01,
        save_strategy="no",
        logging_strategy="epoch",
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=compute_metrics,
    )

    start = time.time()
    trainer.train()
    end = time.time()

    metrics = trainer.evaluate()
    metrics["training_time"] = end - start
    return model_name, metrics

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (where label == -100)
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Use get_entities directly from seqeval.metrics
    results = evaluate.load("seqeval").compute(predictions=true_predictions, references=true_labels, scheme="IOBES") # Added scheme="IOBES"

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

ModuleNotFoundError: No module named 'seqeval'

Compare All Models

In [25]:
models = {
    "XLM-R": "xlm-roberta-base",
    "AfroXLM-R": "Davlan/afro-xlmr-base",
    "DistilBERT": "Davlan/distilbert-base-multilingual-cased-ner-hrl",
    "mBERT": "bert-base-multilingual-cased"
}

results = {}

for name, checkpoint in models.items():
    print(f"\n🔧 Training {name}...")
    model_name, metrics = fine_tune_model(checkpoint, name, dataset)
    results[model_name] = metrics



🔧 Training XLM-R...


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-23-3019213020.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


...

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



🔧 Training DistilBERT...


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

RuntimeError: Error(s) in loading state_dict for Linear:
	size mismatch for bias: copying a param with shape torch.Size([9]) from checkpoint, the shape in current model is torch.Size([1]).

Print & Compare Results

In [26]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset
import evaluate
import numpy as np
import time
# from seqeval.metrics import get_entities # Import get_entities - Removed unnecessary import

def read_conll(file_path):
    sentences = []
    tokens, labels = [], []

    with open(file_path, encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == "":
                if tokens:
                    sentences.append({"tokens": tokens, "ner_tags": labels})
                    tokens, labels = [], []
            else:
                splits = line.split('\t')
                if len(splits) == 2:
                    token, label = splits
                    tokens.append(token)
                    labels.append(label)

    return sentences

# Load and split once
# dataset = Dataset.from_list(read_conll("ner_amharic_sample.conll"))
# dataset = dataset.train_test_split(test_size=0.2)

# Reuse label mapping
# label_list = sorted(set(tag for row in dataset['train'] for tag in row["ner_tags"]))
# label2id = {l: i for i, l in enumerate(label_list)}
# id2label = {i: l for l, i in label2id.items()}


def fine_tune_model(model_checkpoint, model_name, dataset):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    def align_labels_with_tokens(examples): # Changed 'example' to 'examples' to reflect batched input
        tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True, truncation=True)
        aligned_labels = []
        for i, label in enumerate(examples["ner_tags"]): # Iterate through each sentence in the batch
            word_ids = tokenized_inputs.word_ids(batch_index=i) # Get word_ids for the current sentence
            current_aligned_labels = []
            previous_word = None
            for word_id in word_ids:
                if word_id is None:
                    current_aligned_labels.append(-100)
                elif word_id != previous_word:
                    current_aligned_labels.append(label2id[label[word_id]]) # Access label for the current sentence
                else:
                    current_aligned_labels.append(label2id[label[word_id]] if label[word_id].startswith("I-") else -100) # Access label for the current sentence
                previous_word = word_id
            aligned_labels.append(current_aligned_labels)
        tokenized_inputs["labels"] = aligned_labels
        return tokenized_inputs

    tokenized_dataset = dataset.map(align_labels_with_tokens, batched=True)

    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True # Added ignore_mismatched_sizes=True
    )

    args = TrainingArguments(
        output_dir=f"results/{model_name}",
        eval_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=5,
        weight_decay=0.01,
        save_strategy="no",
        logging_strategy="epoch",
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=compute_metrics,
    )

    start = time.time()
    trainer.train()
    end = time.time()

    metrics = trainer.evaluate()
    metrics["training_time"] = end - start
    return model_name, metrics

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (where label == -100)
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = evaluate.load("seqeval").compute(predictions=true_predictions, references=true_labels, scheme="IOBES")

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [17]:
%pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=5b24e8949dda4d29c0a8fb45530030a583e858f29d37092887d63ecf3f3929e9
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [27]:
models = {
    "XLM-R": "xlm-roberta-base",
    "AfroXLM-R": "Davlan/afro-xlmr-base",
    "DistilBERT": "Davlan/distilbert-base-multilingual-cased-ner-hrl",
    "mBERT": "bert-base-multilingual-cased"
}

results = {}

for name, checkpoint in models.items():
    print(f"\n🔧 Training {name}...")
    model_name, metrics = fine_tune_model(checkpoint, name, dataset)
    results[model_name] = metrics


🔧 Training XLM-R...


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-26-4265543240.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


...

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-26-4265543240.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score ar

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


 Print & Compare Results

In [28]:
import pandas as pd

df = pd.DataFrame(results).T
df = df[["eval_precision", "eval_recall", "eval_f1", "eval_loss", "training_time"]]
print(df.sort_values("eval_f1", ascending=False))


            eval_precision  eval_recall  eval_f1  eval_loss  training_time
XLM-R                  0.0          0.0      0.0        0.0     212.202435
AfroXLM-R              0.0          0.0      0.0        0.0     220.526066
DistilBERT             0.0          0.0      0.0        0.0      92.820800
mBERT                  0.0          0.0      0.0        0.0     126.707680
